In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/voicegender/voice.csv


In [3]:
df=pd.read_csv("/kaggle/input/voicegender/voice.csv")
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [4]:
df.isnull().sum()

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64

결측치 없음

In [5]:
x=df.iloc[:,:-1]#마지막 열(label)만 제외한 부분
y=df.iloc[:,-1]


In [6]:
from sklearn.preprocessing import LabelEncoder
#라벨 인코더 생성
encoder =LabelEncoder()
#x를 이용피팅하고 라벨숫자로 변환한다.
y=encoder.fit_transform(y)
y

array([1, 1, 1, ..., 0, 0, 0])

y.head()로 적으면 오류남. dataframe이 아니라 array라서 

In [7]:
# 데이터 표준화 작업
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x_data = scaler.transform(x)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
# df의 20퍼센트를 test set으로 선택

# 2. SVM 모델 적용 - kernel 비교
### 1) default hyperparameter을 이용하여 SVM 돌리기

In [9]:
from sklearn.svm import SVC
from sklearn import metrics
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy Score :')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score :
0.6829652996845426


### 2) linear kernel을 이용해 SVM돌리기

In [10]:
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy Score :')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score :
0.9148264984227129


### 3) rbf

In [11]:
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy Score :')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score :
0.6829652996845426


### 4) poly

In [12]:
svc = SVC(kernel='poly')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy Score :')
print(metrics.accuracy_score(y_test,y_pred))
# 정확도 하락 (polynomial kernel은 training dataset을 overfitting해서 일 가능성 존재)

Accuracy Score :
0.5031545741324921


linear방식의 accurcy가 제일 높음

 # 3. c값 조정
 ### 앞서 정확도가 제일 높게 나온 linear kernel 방식을 이용

In [14]:
C = [1, 10, 100, 1000]
for c in C :
    svc = SVC(kernel='linear',C=c)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    print('when C=',c,', Accuracy Score : ')
    print(metrics.accuracy_score(y_test,y_pred))

when C= 1 , Accuracy Score : 
0.9148264984227129
when C= 10 , Accuracy Score : 
0.9605678233438486
when C= 100 , Accuracy Score : 
0.9637223974763407
when C= 1000 , Accuracy Score : 
0.9542586750788643


C가 100일때 정확도가 가장 높게 나옴